In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [3]:
mnist = tf.keras.datasets.mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [4]:
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2565 - accuracy: 0.9275 - val_loss: 0.1326 - val_accuracy: 0.9587
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1128 - accuracy: 0.9676 - val_loss: 0.0926 - val_accuracy: 0.9727
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0771 - accuracy: 0.9770 - val_loss: 0.0923 - val_accuracy: 0.9722
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0594 - accuracy: 0.9821 - val_loss: 0.0879 - val_accuracy: 0.9725
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0450 - accuracy: 0.9862 - val_loss: 0.0708 - val_accuracy: 0.9779
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0350 - accuracy: 0.9889 - val_loss: 0.0701 - val_accuracy: 0.9785
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0295 - accuracy: 0.9906 - val_loss: 0.0876 - val_accuracy:

In [5]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0817 - accuracy: 0.9798


In [6]:
n = 1
old = model.get_weights()

In [9]:
def remove_random_nodes(acc, loss, weights, n, to_test, x_train, y_train, v=0):
    new_loss = loss
    new_acc = acc
    best_score = 0
    best_model = copy.deepcopy(weights)
    for _ in tqdm.trange(to_test):   
        new = copy.deepcopy(weights)
        to_drop = np.random.choice(len(new[1]), n, replace=False)
        for i in to_drop:
            new[0][:,i] = 0
            new[1][i] = 0
            new[2][i,:] = 0
        model.set_weights(new)
        new_loss, new_acc = model.evaluate(x_train, y_train, verbose=v)
        score = (1 - (new_loss / loss)) + ((new_acc / acc) - 1)
        if best_score < score:
            best_score = score
            best_model = copy.deepcopy(new)
            nodes_removed = to_drop.copy()
    return best_model, best_score, nodes_removed

In [11]:
to_check = 100
new_loss = loss
new_acc = acc
best_score = 0
best_model = copy.deepcopy(old)

for i in tqdm.trange(to_check):   
    new = copy.deepcopy(old)
    #for i in range(len(old)):
    #    new[i] = old[i].copy()
    to_drop = np.random.choice(len(new[1]), n, replace=False)
    # for i in to_drop:
    new[0][:,i] = 0
    new[1][i] = 0
    new[2][i,:] = 0
    model.set_weights(new)
    new_loss, new_acc = model.evaluate(x_test, y_test, verbose=0)
    score = (1 - (new_loss / loss)) + ((new_acc / acc) - 1)
    score1 = (1 - (new_loss / loss))
    score2 = ((new_acc / acc) - 1)
    if best_score < score1 or best_score < score2:
        best_score = score1
        best_model = copy.deepcopy(new)
    if best_score < score2:
        best_score = score2
        best_model = copy.deepcopy(new)
    check = check + 1

100%|██████████| 100/100 [00:36<00:00,  2.74it/s]


In [12]:
best_model[1]

array([ 0.13777253,  0.00683018, -0.04688663, -0.0542522 , -0.02874556,
       -0.06947239,  0.26529318, -0.05660995, -0.12442102, -0.00682503,
       -0.06165336,  0.04902135, -0.06425561,  0.1148253 ,  0.10713325,
       -0.03243035,  0.13046843, -0.02118045,  0.23898   ,  0.012234  ,
       -0.00759443,  0.0615023 ,  0.19653296,  0.0889185 ,  0.11125184,
        0.01821019, -0.06034907, -0.01788706, -0.22678226,  0.25768587,
       -0.00749901, -0.15829737,  0.08297709,  0.07035011,  0.19038919,
        0.21002766,  0.0498327 , -0.16142973, -0.02738184, -0.0705651 ,
       -0.05639892,  0.14290817, -0.04814705, -0.05610737,  0.02354445,
       -0.03746689,  0.07436547,  0.04749129,  0.00861628, -0.00861224,
       -0.02170092,  0.09200769,  0.15906057, -0.13180414, -0.07231998,
       -0.01664761, -0.00881143,  0.10944258,  0.00742158, -0.10521833,
        0.11596415, -0.00629094,  0.04692442,  0.00971017,  0.01094389,
       -0.02010831,  0.13656658,  0.06641621, -0.1453644 , -0.01

In [13]:
model.set_weights(best_model)
loss, acc = model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0789 - accuracy: 0.9797


In [17]:
best_score = 0
best_model = copy.deepcopy(old)
to_test = 25
for i in range(1, 65):
    temp_model, temp_score, nodes_removed = remove_random_nodes(acc, loss, old, i, to_test, x_train, y_train)
    if temp_score > best_score:
        best_model = temp_model
        best_score = temp_score
        print("Found new best model")

  4%|▍         | 1/25 [00:02<01:10,  2.93s/it]


KeyboardInterrupt: 

In [18]:
best_model[1]

array([ 0.13777253,  0.00683018, -0.04688663, -0.0542522 , -0.02874556,
       -0.06947239,  0.26529318, -0.05660995, -0.12442102, -0.00682503,
       -0.06165336,  0.04902135, -0.06425561,  0.1148253 ,  0.10713325,
       -0.03243035,  0.13046843, -0.02118045,  0.23898   ,  0.012234  ,
       -0.00759443,  0.0615023 ,  0.19653296,  0.0889185 ,  0.11125184,
        0.01821019, -0.06034907, -0.01788706, -0.22678226,  0.25768587,
       -0.00749901, -0.15829737,  0.08297709,  0.07035011,  0.19038919,
        0.21002766,  0.0498327 , -0.16142973, -0.02738184, -0.0705651 ,
       -0.05639892,  0.14290817, -0.04814705, -0.05610737,  0.02354445,
       -0.03746689,  0.07436547,  0.04749129,  0.00861628, -0.00861224,
       -0.02170092,  0.09200769,  0.15906057, -0.13180414, -0.07231998,
       -0.01664761, -0.00881143,  0.10944258,  0.00742158, -0.10521833,
        0.11596415, -0.00629094,  0.04692442,  0.00971017,  0.01094389,
       -0.02010831,  0.13656658,  0.06641621, -0.1453644 , -0.01